<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Data Preprocessing </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;padding-left:3em;padding-right:3em;"> The goal of this project is to predict which previous purchased products will be reordered in the latest order of the users in the test set (I call them as the test users in the following discussion). There are 75,000 test users. I divided the whole dataset into train and test data. For the test data, the feature variables were extracted from the prior transaction history and the general information of previous purchased products of the test users, which will be fed into a well-trained model to predict the reordered products in the latest order of the test users.  
<br \>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Suppose the order sequence of a user is ranked as 1, 2, 3, …., n-2, n-1, n,  where n is the latest order. For the train data, I utilized the transaction history before the (n-1)-th order of the test users (not include the (n-1)-th order) and the transaction history before the n-th order of the non-test users to extract feature variables. The output of the train data would be whether the previous purchased products are reordered in the (n-1)-th order of the test users or in the n-th order of the non-test users. The train data will be trained by a neural network in the next stage. 
 </span></div>

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prior_train=pd.read_csv('prior_train_final.csv')

<div style="text-align:left"><span style="color:blue; font-family:Times New Roman; font-size:1.8em;">  
The train data preparation  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The train data involves the test users and the non-test users. The users are selected as the test users via eval_set='test' from the preprocessed table (prior_train). Once the test users are chosen, all the purchased items before the (n-1)-th order for a given test user are selected. The feature input of a given purchased product for a given test user is built upon the interaction between the item and the user as well as 
the general information of the item and the user. The output of a given purchased product for a given test user is whether the product is purchased in the (n-1)-th order of the user. I call the output as ‘reordered’.  
 </span></div>

In [36]:
test_users=prior_train[prior_train.eval_set=='test'].user_id.values
X_test_users=pd.DataFrame(test_users, columns=['user_id'])
prior_train_test_users=pd.merge(X_test_users, prior_train, on='user_id', how='left')
total_orders_user=prior_train_test_users[prior_train_test_users.eval_set=='prior'].groupby('user_id')['order_number'].max()
prior_train_test_users['total_orders_user']=prior_train_test_users.user_id.map(total_orders_user)
X_train_test_users=prior_train_test_users[prior_train_test_users.order_number < prior_train_test_users.total_orders_user]
## build 'user_id','product_id'
print 'build user_id,product_id'
userid_productid=X_train_test_users.groupby(['user_id','product_id'])['order_number'].size().reset_index()
X_test_users=pd.merge(X_test_users,userid_productid[['user_id','product_id']], on='user_id', how='left')
#### reorder or not in the last order as Y output
Y_train_test_users=prior_train_test_users[(prior_train_test_users.order_number ==\
                                          prior_train_test_users.total_orders_user)&\
                                          (prior_train_test_users.reordered==1)][['user_id','product_id', 'reordered']]
X_test_users=pd.merge(X_test_users, Y_train_test_users, on=['user_id','product_id'], how='left').fillna(0)


build user_id,product_id


<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The features for a given purchased product of a given user are as follows:
<br \>
total_orders_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp; Total orders for the user. 
<br \>
unique_items_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total unique products for the user.
<br \>
total_items_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total products for the user.
<br \>
average_items_per_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Average items per order for the user.
<br \>
average_days_per_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Average days between each order for the user.
<br \>
appear_rate&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Appear rate for the product in the user baskets.
<br \>
reorder_num&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total reorder times for the product purchased by the user.
<br \>
add_to_cart_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Mean order of adding the product to the cart by the user.
<br \>
total_order_minus_last_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Order interval between the last order of the product and the (n-1)-th order of the test user (the n-th order of the non-test user). 
<br \>
last_order_ratio&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The ratio of the last order of the product to the (n-1)-th order of the test user (the n-th order of the non-test user). 
<br \>
aisle_id&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Aisle ID of the product.
<br \>
department_id&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Department ID of the product.
<br \>
user_num_product&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the users who buy the product among all users.
<br \>
product_total_orders&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the orders who contain the product in all users.
<br \>
product_total_reorders&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the re-orders who contain the product in all users.
<br \>
reorder_ratio&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The ratio of product_total_reorders to product_total_orders.
<br \>
last_kst_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;k is 1, 2, ..., 19, 20. Whether the product is purchased in the (n-1-k)-th order of the test user (the (n-k)-th order of the non-test user). It is a Boolean value.
<br \>
last_five_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_5st_order
<br \>
last_10_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_10st_order
<br \>
last_15_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_15st_order
<br \>
last_20_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_20st_order
<br \>
I created a function (transform_data) to transform the data in the preprocessed table (prior_train) to the feature variables for the test users.  
 </span></div>

In [ ]:
def transform_data(df_X_train,X_train, status):
    print X_train.shape
    #### total orders for an individual user
    print 'total orders for an individual user'
    total_orders_user=df_X_train.groupby('user_id')['order_number'].max()
    X_train['total_orders_user']=X_train.user_id.map(total_orders_user)
    #### total unique products for an individual user
    print 'total unique products for an individual user'
    def cal1(x):
        return x.nunique()
    unique_items_user=df_X_train.groupby('user_id')['product_id'].apply(cal1)
    X_train['unique_items_user']=X_train.user_id.map(unique_items_user)
    #### total items for an individual user
    print 'total items for an individual user'
    total_items_user=df_X_train.groupby('user_id')['product_id'].size()
    X_train['total_items_user']=X_train.user_id.map(total_items_user)
    #### average items per order for an individual user
    X_train['average_items_per_order']=X_train.total_items_user/X_train.total_orders_user
    #### average days between each order for an individual user
    print 'average days between each order for an individual user'
    days_per_order=df_X_train.groupby(['user_id','order_number'])['days_since_prior_order'].agg('mean')
    average_days_per_order=days_per_order.dropna().reset_index()
    average_days_per_order=average_days_per_order.groupby(['user_id'])['days_since_prior_order'].mean()
    X_train['average_days_per_order']=X_train.user_id.map(average_days_per_order)
    #### appear rate for this product in the user baskets
    print 'appear rate for this product in the user baskets'
    appear_num=df_X_train.groupby(['user_id','product_id'])['order_number'].agg('count').reset_index()
    appear_num=appear_num.rename(columns={'order_number':'appear_num'})
    X_train=pd.merge(X_train, appear_num, on=['user_id','product_id'], how='left')
    X_train['appear_rate']=X_train.appear_num/X_train.total_orders_user
    X_train.drop('appear_num', axis=1, inplace=True)
    #### total reorder number for this product for this user
    print 'total reorder number for this product for this user'
    reorder_num=df_X_train.groupby(['user_id','product_id'])['reordered'].agg([('reorder_num', 'sum')]).reset_index()
    X_train=pd.merge(X_train, reorder_num, on=['user_id','product_id'], how='left')
    #### mean values of add to cart order for this product for this user
    print 'mean values of add to cart order for this product for this user'
    add_to_cart_order_mean=df_X_train.groupby(['user_id','product_id'])['add_to_cart_order'].agg('mean').reset_index()
    X_train=pd.merge(X_train, add_to_cart_order_mean, on=['user_id','product_id'], how='left')
    #### order interval for this product for this user
    print 'order interval for this product for this user'
    def cal2(x):
        if x.nunique()==1:
            return x.max()
        else:
            return x.unique()[-2]
    order_interval_product=df_X_train.groupby(['user_id','product_id'])['order_number'].\
    agg({'last_order':max,'last_second_order':cal2}).reset_index()
    X_train=pd.merge(X_train, order_interval_product, on=['user_id','product_id'], how='left')
    X_train['total_order_minus_last_order']=X_train['total_orders_user']-X_train['last_order']
    #### last order of this product / total orders
    X_train['last_order_ratio']=X_train['last_order'].values / np.array(X_train['total_orders_user'].values, dtype='float')
    X_train.drop(['last_second_order','last_order'], axis=1, inplace=True)
    #### 'aisle_id','department_id' for a product
    print 'aisle_id,department_id for a product'
    aisle_dep=df_X_train.groupby(['user_id','product_id'])['aisle_id','department_id'].agg('mean').reset_index()
    X_train=pd.merge(X_train, aisle_dep, on=['user_id','product_id'], how='left')
    #### the user number who bought this product
    print 'the user number who bought this product'
    def cal3(x):
        return x.nunique()
    user_num_product=df_X_train.groupby('product_id')['user_id'].apply(cal3)
    X_train['user_num_product']=X_train.product_id.map(user_num_product)
    #### orders which have this product
    print 'orders which have this product'
    product_total_orders=df_X_train.groupby('product_id')['user_id'].size()
    X_train['product_total_orders']=X_train.product_id.map(product_total_orders)
    #### re-orders which have this product
    print 're-orders which have this product'
    product_total_reorders=df_X_train[df_X_train.reordered==1].groupby('product_id')['user_id'].size()
    X_train['product_total_reorders']=X_train.product_id.map(product_total_reorders)
    X_train['reorder_ratio']=X_train['product_total_reorders']/X_train['product_total_orders']
    #### order sequence for this product (one hot encoder) 
    print 'order sequence (one hot encoder) for this product (last 20 orders for this user)'
    df_X_train['total_order_minus_order_number']=df_X_train.total_orders_user - df_X_train.order_number
    for i in range(1,21):
        order_sequence_binary=df_X_train[df_X_train.\
                                         total_order_minus_order_number==i][['user_id','product_id','total_order_minus_order_number']]
        order_sequence_binary['total_order_minus_order_number']=order_sequence_binary.total_order_minus_order_number / i
        order_sequence_binary=order_sequence_binary.rename(columns={'total_order_minus_order_number':'last_{}st_order'.format(i)})
        X_train=pd.merge(X_train, order_sequence_binary, on=['user_id','product_id'], how='left').fillna(0)
    if status == 'train':
        X_train['last_five_orders_sum']=X_train.ix[:,21:26].sum(1)
        X_train['last_10_orders_sum']=X_train.ix[:,21:31].sum(1)
        X_train['last_15_orders_sum']=X_train.ix[:,21:36].sum(1)
        X_train['last_20_orders_sum']=X_train.ix[:,21:41].sum(1) 
    else: 
        X_train['last_five_orders_sum']=X_train.ix[:,20:25].sum(1)
        X_train['last_10_orders_sum']=X_train.ix[:,20:30].sum(1)
        X_train['last_15_orders_sum']=X_train.ix[:,20:35].sum(1)
        X_train['last_20_orders_sum']=X_train.ix[:,20:40].sum(1) 
    print X_train.shape
    return X_train

In [ ]:
X_test_users=transform_data(X_train_test_users,X_test_users,'train')

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The train data only involving the test users will be suffered to the imbalanced issue.  Almost 90% of the data belongs to the class 0 (reordered=0) can cause 90% accuracy immediately while being trained by a classification model.  This is because the learning algorithm of a model tends to be biased towards the majority class and classifies every example as the majority class.
<br \>
So what is my strategy  when the data is imbalanced?
<br \>
1)	Oversample the minority class.
<br \>
2)	Undersample the majority class. 
<br \>
3)	Adjust the class weight in the learning algorithm. (discussed in the next stage)
<br \>
In order to increase the sample number of the minority class, I included the non-test users into the train data.  The non-test users are selected via eval_set='train' from the preprocessed table (prior_train). I only considered the purchased products that are reordered in the n-th order of the non-test users. Therefore, the ‘reordered’ output of these products are all ‘True’ (1). The ‘transform_data’ function transforms the information of these selected purchased products to the feature variables for the non-test users. 
 </span></div>

In [60]:
userid_productid=prior_train[(prior_train.eval_set=='train')&(prior_train.reordered==1)][['user_id','product_id','reordered']].reset_index()
X_train_users=userid_productid[['user_id','product_id','reordered']]
train_users=pd.DataFrame(X_train_users.user_id.unique(), columns=['user_id'])
prior_train_selected=pd.merge(train_users, prior_train[prior_train.eval_set=='prior'], on='user_id', how='left')
total_orders_user=prior_train.groupby('user_id')['order_number'].max()
prior_train_selected['total_orders_user']=prior_train_selected.user_id.map(total_orders_user)

In [ ]:
X_train_users=transform_data(prior_train_selected,X_train_users,'train')

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
In order to undersample the majority class (‘reordered’=0) of the train data, I deleted most of the samples that the products don't appear within last 20 orders, and remained only one for each test user. This is because if a product is not purchased within the last 20 orders of a given user, the chance of getting reordered (‘reordered’=1) in the latest order is tiny. This method substantially decreases the sample number of the class 0.

 </span></div>

In [ ]:
reordered_0=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==0)]
reordered_1=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==1)]
X_test_users=X_test_users[X_test_users.last_20_orders_sum!=0]
reordered_0_seleted=reordered_0.groupby(['user_id'])['product_id'].min().reset_index()
reordered_0=pd.merge(reordered_0_seleted, reordered_0, on=['user_id', 'product_id'], how='left')
X_test_users=pd.concat([X_test_users, reordered_0, reordered_1],ignore_index=True)

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
Finally, the feature variables and output variables from the test user and non-test users are combined to create the train data and saved in ‘X.csv’ for the next stage of data modeling. 
The percentages of the class 0 (‘reordered’=0) and class 1 (‘reordered’=1) in the train data are 72.6% and 27.4 %.  
 </span></div>

In [67]:
X=pd.concat([X_test_users, X_train_users],ignore_index=True)
X.to_csv('X.csv')

In [5]:
print X.shape[0]
print np.bincount(np.array(X.reordered.values, dtype='int'))
print np.bincount(np.array(X.reordered.values, dtype='int'))/float(X.shape[0])

4690951
[3404649 1286302]
[ 0.72579078  0.27420922]


<div style="text-align:left"><span style="color:blue; font-family:Times New Roman; font-size:1.8em;">  
The test data preparation  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">For the test data, the feature variables are extracted from the transaction history before the n-th order of the test users. The output of whether a purchased product is reordered in the n-th order of a given test user will be predicted by a trained model. The features of the test data are created by the ‘transform_data’ function and saved as ‘X_test.csv’. 
 </span></div>

In [8]:
test_users=prior_train[prior_train.eval_set=='test'].user_id.values
X_test_users=pd.DataFrame(test_users, columns=['user_id'])
prior_train_test_users=pd.merge(X_test_users, prior_train, on='user_id', how='left')
total_orders_user=prior_train_test_users.groupby('user_id')['order_number'].max()
prior_train_test_users['total_orders_user']=prior_train_test_users.user_id.map(total_orders_user)
prior_train_test_users=prior_train_test_users[prior_train_test_users.eval_set=='prior']
## build 'user_id','product_id'
print 'build user_id,product_id'
userid_productid=prior_train_test_users.groupby(['user_id','product_id'])['order_number'].size().reset_index()
X_test_users=pd.merge(X_test_users,userid_productid[['user_id','product_id']], on='user_id', how='left')

In [ ]:
X_test=transform_data(prior_train_test_users,X_test_users,'test')

In [19]:
X_test.to_csv('X_test.csv')